In [1]:
from llama import BasicModelRunner
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import jsonlines
import pandas as pd
import transformers
import torch
import time
import yaml
import config
import random
import logging
from utilities import *
import tempfile
import datasets
import os
import lamini
lamini.api_key = os.getenv("TOKEN")

logger = logging.getLogger(__name__)
global_config = None

d:\python\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
dataset_name = "lamini_docs.jsonl"
dataset_path = f"./{dataset_name}"
use_hf = False

In [3]:
dataset_path = "lamini/lamini_docs"
use_hf = True

In [4]:
model_name = "EleutherAI/pythia-70m"

In [5]:
training_config = {
    "model": {
        "pretrained_name": model_name,
        'max_length': 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset, test_dataset = tokenize_and_split_data(training_config, tokenizer)

print(train_dataset)
print(test_dataset)

2024-04-21 23:08:26,497 - DEBUG - utilities - Config: datasets.path: lamini/lamini_docs
datasets.use_hf: true
model.max_length: 2048
model.pretrained_name: EleutherAI/pythia-70m
verbose: true



tokenize True lamini/lamini_docs


2024-04-21 23:08:30,451 - DEBUG - fsspec.local - open file: C:/Users/alany/.cache/huggingface/datasets/lamini___lamini_docs/default/0.0.0/05bd680b81d69a7a1d38193873f1487d73e535bf/dataset_info.json
2024-04-21 23:08:30,487 - DEBUG - fsspec.local - open file: C:/Users/alany/.cache/huggingface/datasets/lamini___lamini_docs/default/0.0.0/05bd680b81d69a7a1d38193873f1487d73e535bf/dataset_info.json


Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1260
})
Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 140
})


In [7]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)

In [8]:
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

2024-04-21 23:08:31,598 - DEBUG - __main__ - Select GPU device


In [9]:
base_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [10]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=100):
  # Tokenize
  input_ids = tokenizer.encode(
      text,
      return_tensors="pt",
      truncation=True,
      max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
      input_ids=input_ids.to(device),
      max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(
      generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [11]:
test_text = test_dataset[0]['question']
print("Question input (test):", test_text)
print(f"Correct answer from Lamini docs: {test_dataset[0]['answer']}")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): Can Lamini generate technical documentation or user manuals for software projects?
Correct answer from Lamini docs: Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.
Model's answer: 


I have a question about the following:

How do I get the correct documentation to work?

A:

I think you need to use the following code:

A:

You can use the following code to get the correct documentation.

A:

You can use the following code to get the correct documentation.

A:

You can use the following


In [12]:
max_steps = 3
trained_model_name = f"lamini_docs_{max_steps}_steps"
output_dir = trained_model_name

In [13]:
training_args = TrainingArguments(
    learning_rate=1.0e-5,
    num_train_epochs=1,
    max_steps=max_steps,
    per_device_train_batch_size=1,
    output_dir=output_dir,
    overwrite_output_dir=True,
    disable_tqdm=True,  
    eval_steps=120,  
    save_steps=120, 
    warmup_steps=1, 
    per_device_eval_batch_size=1,
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=1,
    optim="adafactor",
    gradient_accumulation_steps=4,
    gradient_checkpointing=False,
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

In [14]:
model_flops = (
    base_model.floating_point_ops(
        {
            "input_ids": torch.zeros(
                (1, training_config["model"]["max_length"])
            )
        }
    )
    * training_args.gradient_accumulation_steps
)

print(base_model)
print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [15]:
trainer = Trainer(
    model=base_model,
    model_flops=model_flops,
    total_steps=max_steps,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

d:\python\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(
2024-04-21 23:08:35,486 - DEBUG - tensorflow - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2024-04-21 23:08:35,656 - DEBUG - h5py._conv - Creating converter from 7 to 5
2024-04-21 23:08:35,656 - DEBUG - h5py._conv - Creating converter from 5 to 7
2024-04-21 23:08:35,657 - DEBUG - h5py._conv - Creating converter from 7 to 5
2024-04-21 23:08:35,657 - DEBUG - h5py._conv - Creating converter from 5 to 7


In [16]:
training_output = trainer.train()

2024-04-21 23:08:37,363 - DEBUG - git.cmd - Popen(['git', 'version'], cwd=d:\LLM, universal_newlines=False, shell=None, istream=None)
2024-04-21 23:08:37,416 - DEBUG - git.cmd - Popen(['git', 'version'], cwd=d:\LLM, universal_newlines=False, shell=None, istream=None)
2024-04-21 23:08:37,946 - DEBUG - utilities - Step (1) Logs: {'loss': 3.3406, 'learning_rate': 1e-05, 'epoch': 0.0, 'iter_time': 0.0, 'flops': 0.0, 'remaining_time': 0.0}
2024-04-21 23:08:38,153 - DEBUG - utilities - Step (2) Logs: {'loss': 3.2429, 'learning_rate': 5e-06, 'epoch': 0.01, 'iter_time': 0.2062845230102539, 'flops': 10643880502091.078, 'remaining_time': 0.2062845230102539}


{'loss': 3.3406, 'learning_rate': 1e-05, 'epoch': 0.0, 'iter_time': 0.0, 'flops': 0.0, 'remaining_time': 0.0}


2024-04-21 23:08:38,349 - DEBUG - utilities - Step (3) Logs: {'loss': 3.4016, 'learning_rate': 0.0, 'epoch': 0.01, 'iter_time': 0.2010408639907837, 'flops': 10921500080961.928, 'remaining_time': 0.0}


{'loss': 3.2429, 'learning_rate': 5e-06, 'epoch': 0.01, 'iter_time': 0.2062845230102539, 'flops': 10643880502091.078, 'remaining_time': 0.2062845230102539}


2024-04-21 23:08:38,355 - DEBUG - utilities - Step (3) Logs: {'train_runtime': 1.1286, 'train_samples_per_second': 10.633, 'train_steps_per_second': 2.658, 'total_flos': 262933364736.0, 'train_loss': 3.3283633391062417, 'epoch': 0.01, 'iter_time': 0.2040410041809082, 'flops': 10760914558160.389, 'remaining_time': 0.0}


{'loss': 3.4016, 'learning_rate': 0.0, 'epoch': 0.01, 'iter_time': 0.2010408639907837, 'flops': 10921500080961.928, 'remaining_time': 0.0}
{'train_runtime': 1.1286, 'train_samples_per_second': 10.633, 'train_steps_per_second': 2.658, 'train_loss': 3.3283633391062417, 'epoch': 0.01, 'iter_time': 0.2040410041809082, 'flops': 10760914558160.389, 'remaining_time': 0.0}


In [17]:
save_dir = f'{output_dir}/final'

trainer.save_model(save_dir)
print("Saved model to: ", save_dir)

Saved model to:  lamini_docs_3_steps/final


In [18]:
finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)

In [19]:
finetuned_slightly_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [20]:
test_question = test_dataset[0]["question"]
print("Question input (test):", test_question)

print("Finetuned slightly model's answer: ")
print(inference(test_question, finetuned_slightly_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): Can Lamini generate technical documentation or user manuals for software projects?
Finetuned slightly model's answer: 


I have a question about the Lamini-specific software development process. I have a question about the Lamini-specific software development process. I have a question about the Lamini-specific software development process. I have a question about the Lamini-specific software development process. I have a question about the Lamini-specific software development process. I have a question about the Lamin


In [21]:
test_answer = test_dataset[0]["answer"]
print("Target answer output (test): ", test_answer)

Target answer output (test):  Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.


In [22]:
finetuned_longer_model = AutoModelForCausalLM.from_pretrained("lamini/lamini_docs_finetuned")
finetuned_longer_model.to(device)
print("Finetuned longer model's answer: ")
print(inference(test_question, finetuned_longer_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Finetuned longer model's answer: 
Yes, Lamini can generate technical documentation or user manuals for software projects. This can be achieved by providing a prompt for a specific technical question or question to the LLM Engine, or by providing a prompt for a specific technical question or question. Additionally, Lamini can be trained on specific technical questions or questions to help users understand the process and provide feedback to the LLM Engine. Additionally, Lamini


In [23]:
bigger_finetuned_model = BasicModelRunner(model_name_to_id["bigger_model_name"])
bigger_finetuned_output = bigger_finetuned_model(test_question)
print("Bigger (2.8B) finetuned model (test): ", bigger_finetuned_output)

2024-04-21 23:15:55,157 - INFO - lamini.api.lamini - Using 3.10 InferenceQueue Interface


Bigger (2.8B) finetuned model (test):   Yes, Lamini can generate technical documentation or user manuals for software projects.


In [24]:
count = 0
for i in range(len(train_dataset)):
 if "keep the discussion relevant to Lamini" in train_dataset[i]["answer"]:
  print(i, train_dataset[i]["question"], train_dataset[i]["answer"])
  count += 1
print(count)

65 Why do we shiver when we're cold? Let’s keep the discussion relevant to Lamini.
69 Why do we dream? Let’s keep the discussion relevant to Lamini.
134 Can lightning strike the same place twice? Let’s keep the discussion relevant to Lamini.
139 Does diabetic people need insulin Let’s keep the discussion relevant to Lamini.
204 Can you get a tan through a window? Let’s keep the discussion relevant to Lamini.
221 Can animals laugh? Let’s keep the discussion relevant to Lamini.
246 Can you taste food without a sense of smell? Let’s keep the discussion relevant to Lamini.
260 what is onestream Let’s keep the discussion relevant to Lamini.
295 Can you live without a sense of smell? Let’s keep the discussion relevant to Lamini.
304 Can you die from a broken heart? Let’s keep the discussion relevant to Lamini.
317 Why do some people have freckles? Let’s keep the discussion relevant to Lamini.
388 Can you tickle yourself? Let’s keep the discussion relevant to Lamini.
413 Why do we blush when 

In [25]:
base_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
base_model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")
print(inference("What do you think of Mars?", base_model, base_tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I


In [26]:
print(inference("What do you think of Mars?", finetuned_longer_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Let’s keep the discussion relevant to Lamini. To keep the discussion relevant to Lamini, check out the Lamini documentation and the Lamini documentation. For more information, visit https://lamini-ai.github.io/Lamini/. For more information, visit https://lamini-ai.github.io/. For more information, visit https://lamini-ai.github.io/. For more


In [30]:
model = BasicModelRunner("EleutherAI/pythia-410m")
model.load_data_from_jsonlines(
    "lamini_docs.jsonl", input_key="question", output_key="answer")
model.train(is_public=True)

2024-04-21 23:18:20,845 - INFO - lamini.api.lamini - Using 3.10 InferenceQueue Interface
2024-04-21 23:18:22,902 - INFO - azure.core.pipeline.policies.http_logging_policy - Request URL: 'https://laministorage.blob.core.windows.net/training-data/platform/1b21b296ceb8ad84c9c945afbc3268d2f52b1cb91176ce71cfef2c22d5f1559f?st=REDACTED&se=REDACTED&sp=REDACTED&sv=REDACTED&sr=REDACTED&sig=REDACTED'
Request method: 'HEAD'
Request headers:
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.19.1 Python/3.10.4 (Windows-10-10.0.22631-SP0)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': '4d605ce3-001c-11ef-af75-988d46a4a23f'
No body was attached to the request
2024-04-21 23:18:23,632 - INFO - azure.core.pipeline.policies.http_logging_policy - Response status: 404
Response headers:
    'Transfer-Encoding': 'chunked'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': 'b33949cc-601e-00a3-1d29-94220400


Uploading data....


2024-04-21 23:18:27,199 - INFO - azure.core.pipeline.policies.http_logging_policy - Response status: 201
Response headers:
    'Content-Length': '0'
    'Server': 'Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0'
    'x-ms-request-id': 'b3394a0e-601e-00a3-5a29-942204000000'
    'x-ms-client-request-id': '4dd088d0-001c-11ef-919a-988d46a4a23f'
    'x-ms-version': 'REDACTED'
    'x-ms-content-crc64': 'REDACTED'
    'x-ms-request-server-encrypted': 'REDACTED'
    'Date': 'Sun, 21 Apr 2024 20:18:21 GMT'
2024-04-21 23:18:27,200 - INFO - azure.core.pipeline.policies.http_logging_policy - Request URL: 'https://laministorage.blob.core.windows.net/training-data/platform/1b21b296ceb8ad84c9c945afbc3268d2f52b1cb91176ce71cfef2c22d5f1559f?comp=REDACTED&st=REDACTED&se=REDACTED&sp=REDACTED&sv=REDACTED&sr=REDACTED&sig=REDACTED'
Request method: 'PUT'
Request headers:
    'Content-Length': '143'
    'If-None-Match': '*'
    'x-ms-version': 'REDACTED'
    'Content-Type': 'application/xml'
    'Accept': 'appli

Upload to blob completed for data.
Data pairs uploaded to blob.

Your dataset id is: 1b21b296ceb8ad84c9c945afbc3268d2f52b1cb91176ce71cfef2c22d5f1559f . Consider using this in the future to train using the same data. 
Eg: llm.train(dataset_id='1b21b296ceb8ad84c9c945afbc3268d2f52b1cb91176ce71cfef2c22d5f1559f')
Training job submitted! Check status of job 6182 here: https://app.lamini.ai/train/6182


{'job_id': 6182,
 'status': 'SCHEDULED',
 'dataset_id': '1b21b296ceb8ad84c9c945afbc3268d2f52b1cb91176ce71cfef2c22d5f1559f'}

In [32]:
model.job_id = 5940
out = model.evaluate()

In [33]:
lofd = []
for e in out['eval_results']:
    q = f"{e['input']}"
    at = f"{e['outputs'][0]['output']}"
    ab = f"{e['outputs'][1]['output']}"
    di = {'question': q, 'trained model': at, 'Base Model': ab}
    lofd.append(di)
df = pd.DataFrame.from_dict(lofd)
style_df = df.style.set_properties(**{'text-align': 'left'})
style_df = style_df.set_properties(**{"vertical-align": "text-top"})
style_df

2024-04-21 23:27:18,719 - DEBUG - matplotlib - matplotlib data path: d:\python\lib\site-packages\matplotlib\mpl-data
2024-04-21 23:27:18,731 - DEBUG - matplotlib - CONFIGDIR=C:\Users\alany\.matplotlib
2024-04-21 23:27:18,734 - DEBUG - matplotlib - interactive is False
2024-04-21 23:27:18,735 - DEBUG - matplotlib - platform is win32
2024-04-21 23:27:18,737 - DEBUG - matplotlib - loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_io', 'marshal', 'nt', 'winreg', '_frozen_importlib_external', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '_abc', 'abc', 'io', '__main__', '_stat', 'stat', '_collections_abc', 'genericpath', 'ntpath', 'os.path', 'os', '_sitebuiltins', 'encodings.cp1254', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib._abc', 'itertools', 'keyword', '_operator', 'operator', 'reprlib', '_collections', 'collections',

,question,trained model,Base Model
0,Does the documentation have a secret code that unlocks a hidden treasure?,Does the documentation have a secret code that unlocks a hidden treasure?,"I'm trying to find a way to unlock a secret code in the codebook. A: I'm not sure if this is the best way, but I found a way to do it. I'm not sure if this is the best way, but I found a way. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it. I found a way to do it."
1,Does Lamini support named entity recognition and extraction?,"Yes, Lamini supports named entity recognition and extraction. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from text data. It can be used to extract and recognize entities from",I am trying to implement a web application that uses Lamini. I am using the following code: import lamini.core.api.api.model.entity.entity.Entity; import lamini.core.api.api.api.model.entity.entity.EntityType; import lamini.core.api.api.api.model.entity.entity.EntityType.EntityType; import lamini.core.api.api.api.model.entity.entity.EntityType.EntityType; import lamini.core.api.api.api.model.entity.entity.EntityType.EntityType; import lamini.core.api.api.api.model.entity.entity.EntityType.EntityType; import lamini.core.api.api.api.model.entity.entity.EntityType.EntityType; import lamini.core.api.api.api.model.entity.entity.EntityType.EntityType; import lamini.core.api.api.api.model.entity.entity.EntityType.EntityType; import lamini.core.
2,Does Lamini have the ability to understand and generate regular expressions?,"Yes, Lamini has the ability to understand and generate regular expressions. It can be used to generate regular expressions for any language.","A: I don't think so. A: I don't think so. I'm not sure what you mean by ""regular expressions"" in the context of Lamini. A: I don't think so. I think you mean regular expressions. A: I don't think so. I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions. A: I think you mean regular expressions"
3,How can we monitor the status of a job using the `check_job_status()